In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd

In [2]:
# Replace 'your_file.csv' with the name of your CSV file
df = pd.read_csv('C:\\Users\\yacco\\afi_project_1\\afi_final_project\\predictions_output_gradient_boosting.csv')

# Display the first few rows of the data
df.head()

,Job tenure,Age,Car price,Funding amount,Down payment,Loan duration,Monthly payment,Credit event,Married,Homeowner,Group,Predicted_PD,True_Label
0,2,22,8900,8900,0,60,0.084828,0,0,0,0,0.244713,1
1,0,55,10400,9400,0,72,0.083889,0,0,0,0,0.327168,0
2,1,40,15990,14990,0,60,0.127142,0,1,0,0,0.189396,1
3,5,43,19999,23233,0,72,0.153289,0,0,0,0,0.391459,0
4,1,26,11800,4298,1,24,0.115528,0,0,0,0,0.119370,0


In [3]:
df_true = df.drop(columns=['True_Label'])

In [4]:
X = df_true.drop(columns=['Predicted_PD'])  # Features used in the black box model
y_pred = df_true['Predicted_PD']  # Predicted values (lack box model)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y_pred, test_size=0.2, random_state=42)

from sklearn.tree import DecisionTreeRegressor

# Train a decision tree regressor instead of a classifier
tree_model = DecisionTreeRegressor(max_depth=10)
tree_model.fit(X_train, y_train)

# Use the decision tree to transform the data (leaf node predictions)
X_train_leaves = tree_model.apply(X_train)  # Leaf node indices for each sample
X_test_leaves = tree_model.apply(X_test)


In [6]:
X_train_leaves.size

1783

In [7]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler

# Scale the leaf node outputs (optional, helps Lasso regression)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_leaves.reshape(-1, 1))  # Reshape to fit scaling
X_test_scaled = scaler.transform(X_test_leaves.reshape(-1, 1))

# Train Lasso regression model (L1 regularization for sparsity)
lasso_model = Lasso(alpha=0.01)  # Alpha controls the strength of the penalty
lasso_model.fit(X_train_scaled, y_train)

Lasso(alpha=0.01)

In [9]:
X_train_scaled

array([[-1.28166971],
       [-0.2354599 ],
       [ 1.17025829],
       ...,
       [-1.28166971],
       [-1.24531493],
       [ 0.03518124]])

In [10]:
from sklearn.metrics import mean_squared_error, r2_score

# Predict on the test set using the surrogate model
y_pred_surrogate = lasso_model.predict(X_test_scaled)

# Evaluate the surrogate model performance
mse = mean_squared_error(y_test, y_pred_surrogate)
r2 = r2_score(y_test, y_pred_surrogate)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

Mean Squared Error: 0.022946460446525475
R-squared: 0.182457567270356


In [9]:
# Lasso regression coefficients
print("Lasso Regression Coefficients:", lasso_model.coef_)


Lasso Regression Coefficients: [-0.]
